### **Metrics calculation rough work, discovering motivations, and creating additional data for formal metrics analysis later**

In [ ]:
import pandas as pd
import os

In [ ]:
df_baseline = pd.read_csv('baseline_mcq.csv')
df_label_change_mcq = pd.read_csv('label_change_mcq.csv')
df_tf = pd.read_csv('tf_structured.csv')
df_mix = pd.read_csv('mixed_tf_label.csv')
df_2_opt = pd.read_csv('tf_2_opt_model_predictions.csv')

### **Metrics analysis formally**

Categories

In [ ]:
list(df_baseline.category.unique())

['Offensiveness',
 'Unfairness and Bias',
 'Physical Health',
 'Mental Health',
 'Illegal Activities',
 'Ethics and Morality',
 'Privacy and Property']

Calculating correctness metrics (Accuracy, macro Precision, Recall, F1)

In [ ]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def calculate_mcq_metrics(df, group_col='category', gold_col='answer_idx', pred_col='pred_mcq_idx'):
    """
    Calculates precision, recall, f1 (macro), and accuracy per category and overall.
    """
    df = df.dropna(subset=[gold_col, pred_col])

    def compute_stats(y_true, y_pred):
        # We use 'macro' averaging to treat all MCQ choices equally
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0
        )
        accuracy = accuracy_score(y_true, y_pred)
        return pd.Series({
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'accuracy': accuracy,
            'sample_count': len(y_true)
        })

    # 1. Calculate metrics per category
    # We select only the label and prediction columns to keep the group context clean
    category_results = df.groupby(group_col)[[gold_col, pred_col]].apply(
        lambda x: compute_stats(x[gold_col], x[pred_col])
    ).reset_index()

    # 2. Calculate overall metrics
    overall_stats = compute_stats(df[gold_col], df[pred_col])
    overall_stats[group_col] = 'OVERALL'

    # 3. Combine results
    final_report = pd.concat([category_results, pd.DataFrame([overall_stats])], ignore_index=True)
    final_report.index = final_report.index + 1

    # Round for readability
    return final_report.round(4)

Baseline (MCQ and number dot label style)

In [ ]:
temp = calculate_mcq_metrics(df_baseline)
temp.to_csv("metrics/baseline_accuracy_and_more.csv", index=False)
temp

,category,precision,recall,f1_score,accuracy,sample_count
1,Ethics and Morality,0.8467,0.8522,0.8483,0.8666,1934.0
2,Illegal Activities,0.8919,0.8898,0.8907,0.8915,1778.0
3,Mental Health,0.8952,0.8935,0.8941,0.9017,1566.0
4,Offensiveness,0.8868,0.7453,0.7874,0.8283,1805.0
5,Physical Health,0.9196,0.9333,0.9260,0.9278,1149.0
6,Privacy and Property,0.8790,0.8833,0.8809,0.8830,1299.0
7,Unfairness and Bias,0.7788,0.8319,0.7834,0.7337,1904.0
8,OVERALL,0.8594,0.8652,0.8618,0.8551,11435.0


Label change (MCQ, label styles: number dot, alphabet paranthesis, number paranthesis, roman numeral paranthesis)

In [ ]:
temp = pd.concat([df_baseline, df_label_change_mcq], ignore_index=True)
temp = calculate_mcq_metrics(temp, group_col="label_style")
temp.to_csv("metrics/label_change_accuracy_and_more.csv", index = False)
temp

,label_style,precision,recall,f1_score,accuracy,sample_count
1,alpha_paren,0.8634,0.8659,0.8640,0.8560,11435.0
2,num_dot,0.8594,0.8652,0.8618,0.8551,11435.0
3,num_paren,0.8577,0.8623,0.8595,0.8529,11434.0
4,roman_paren,0.8616,0.8654,0.8629,0.8542,11435.0
5,OVERALL,0.8605,0.8647,0.8621,0.8545,45739.0


Computing pair-wise robustness and coherence metrics PC
(Prediction Consistency), LFR (Label Flip Rate), AR (Accuracy Retention), and WCA (Worst-Case Accuracy per seed) for True/False style questions for various option-labels styles

In [ ]:
import pandas as pd
import numpy as np

def tf_structured_metrics_per_category(df: pd.DataFrame, label_name):
    """
    Computes PC, LFR, AR, WCA per category for tf_structured probe.
    Assumptions:
    - Columns: id, category, expected_tf, pred_tf, is_valid, is_correct
    - Each id should have two rows: one expected_tf=True, one expected_tf=False
    - Pair-level metrics use only ids where BOTH rows are valid.
    """

    d = df.copy()
    d = d[d.label_style == label_name]

    # 1. Ensure booleans and handle NaNs
    cols_to_fix = ["expected_tf", "pred_tf", "is_valid", "is_correct"]
    for c in cols_to_fix:
        if c in d.columns:
            # Drop NaNs for these critical columns to avoid errors
            d = d.dropna(subset=[c])
            d[c] = d[c].astype(bool)

    # 2. Keep only valid rows
    d_valid = d[d["is_valid"]]

    def calculate_group_metrics(subset):
        if subset.empty:
            return pd.Series()

        # Pivot: index by 'id', columns by 'expected_tf'
        # This handles the pairs (True/False expected) for each claim ID
        pivot_pred = subset.pivot_table(index="id", columns="expected_tf", values="pred_tf", aggfunc="first")
        pivot_corr = subset.pivot_table(index="id", columns="expected_tf", values="is_correct", aggfunc="first")

        # Ensure both True and False columns exist
        if True not in pivot_pred.columns or False not in pivot_pred.columns:
            return pd.Series({"n_pair_ids": 0})

        # Drop IDs that don't have both rows valid
        valid_pairs_idx = pivot_pred.dropna(subset=[True, False]).index
        pred_T = pivot_pred.loc[valid_pairs_idx, True]
        pred_F = pivot_pred.loc[valid_pairs_idx, False]
        corr_T = pivot_corr.loc[valid_pairs_idx, True]
        corr_F = pivot_corr.loc[valid_pairs_idx, False]

        # Metric Calculations
        PC = (pred_T != pred_F).mean()
        LFR = 1.0 - PC
        both_correct = (corr_T & corr_F)
        WCA = both_correct.mean()

        denom = corr_T.sum()
        AR = (both_correct.sum() / denom) if denom > 0 else np.nan

        return pd.Series({
            "label_style": label_name,
            "n_valid_rows": int(len(subset)),
            "n_pair_ids": int(len(valid_pairs_idx)),
            "PC": PC,
            "LFR": LFR,
            "AR": AR,
            "WCA": WCA,
            "Accuracy": subset["is_correct"].mean()
        })

    # 3. Apply grouping logic
    # This will cover 'Offensiveness', 'Physical Health', etc. automatically
    category_results = d_valid.groupby("category", sort=False).apply(
        lambda x: calculate_group_metrics(x),
        include_groups=False
    ).reset_index()

    # 4. Calculate Overall
    overall_metrics = calculate_group_metrics(d_valid)
    overall_metrics["category"] = "OVERALL"

    # 5. Combine and Format
    report = pd.concat([category_results, pd.DataFrame([overall_metrics])], ignore_index=True)
    report.index = report.index + 1

    return report.round(4)

# Usage:
# results = tf_structured_metrics_per_category(df)
# print(results)

In [ ]:
# Combining metrics from the original tf_structured csv and
# mix_tf csv (the csv that had the tf_structured questions except
# with different option-label styles)
df_tf_multiple_styles = pd.concat([df_tf, df_mix], ignore_index=True)

In [ ]:
# Model inference metrics on T/F style questions and num_dot option-label style
temp = tf_structured_metrics_per_category(df_tf_multiple_styles, label_name="num_dot")
temp.to_csv("metrics/tf_metrics_num_dot.csv", index=False)
temp

,category,label_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
1,Offensiveness,num_dot,3610,1805,0.9247,0.0753,0.9272,0.7900,0.8277
2,Unfairness and Bias,num_dot,3808,1904,0.8997,0.1003,0.8908,0.6597,0.7098
3,Physical Health,num_dot,2298,1149,0.9164,0.0836,0.9790,0.8912,0.9330
4,Mental Health,num_dot,3132,1566,0.8729,0.1271,0.9731,0.8531,0.9167
5,Illegal Activities,num_dot,3555,1777,0.8706,0.1294,0.9678,0.8464,0.9111
6,Ethics and Morality,num_dot,3867,1933,0.8660,0.1340,0.9531,0.8205,0.8875
7,Privacy and Property,num_dot,2598,1299,0.8707,0.1293,0.9613,0.8406,0.9053
8,OVERALL,num_dot,22868,11433,0.8881,0.1119,0.9489,0.8068,0.8627


In [ ]:
# Model inference metrics on T/F style questions and alpha_dot option-label style
temp = tf_structured_metrics_per_category(df_tf_multiple_styles, label_name="alpha_dot")
temp.to_csv("metrics/tf_metrics_alpha_dot.csv", index=False)
temp

,category,label_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
1,Offensiveness,alpha_dot,3610,1805,0.9219,0.0781,0.9271,0.7961,0.8352
2,Unfairness and Bias,alpha_dot,3808,1904,0.8866,0.1134,0.8568,0.6287,0.6854
3,Physical Health,alpha_dot,2298,1149,0.9173,0.0827,0.9845,0.8860,0.9273
4,Mental Health,alpha_dot,3132,1566,0.8736,0.1264,0.9738,0.8538,0.9170
5,Illegal Activities,alpha_dot,3556,1778,0.8847,0.1153,0.9690,0.8611,0.9187
6,Ethics and Morality,alpha_dot,3866,1932,0.8690,0.1310,0.9585,0.8245,0.8901
7,Privacy and Property,alpha_dot,2598,1299,0.8699,0.1301,0.9571,0.8414,0.9065
8,OVERALL,alpha_dot,22868,11433,0.8883,0.1117,0.9454,0.8052,0.8611


In [ ]:
# Model inference metrics on T/F style questions and roman_dot option-label style
temp = tf_structured_metrics_per_category(df_tf_multiple_styles, label_name="roman_dot")
temp.to_csv("metrics/tf_metrics_roman_dot.csv", index=False)
temp

,category,label_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
1,Offensiveness,roman_dot,3610,1805,0.9191,0.0809,0.9274,0.7856,0.8260
2,Unfairness and Bias,roman_dot,3808,1904,0.8981,0.1019,0.8739,0.6555,0.7064
3,Physical Health,roman_dot,2298,1149,0.9164,0.0836,0.9789,0.8886,0.9304
4,Mental Health,roman_dot,3132,1566,0.8678,0.1322,0.9751,0.8487,0.9148
5,Illegal Activities,roman_dot,3556,1778,0.8583,0.1417,0.9662,0.8369,0.9078
6,Ethics and Morality,roman_dot,3867,1933,0.8795,0.1205,0.9600,0.8308,0.8911
7,Privacy and Property,roman_dot,2598,1299,0.8614,0.1386,0.9633,0.8291,0.8984
8,OVERALL,roman_dot,22869,11434,0.8856,0.1144,0.9477,0.8035,0.8607


Single sentence true/false (made from specific subset of mcqs having 2 mcq options) metrics calculation

In [ ]:
import pandas as pd
import numpy as np

def calculate_metrics_2_options_tf(df: pd.DataFrame):
    """
    Computes PC, LFR, AR, and WCA for the 2-options True/False probe.

    Columns expected: id, question, gold_answer, model_output, predicted_answer, correct, [category]
    ID logic: base_id * 100 + (0 for correct option, 1 for wrong option)
    """

    d = df.copy()

    # 1. Decompose IDs
    d['base_id'] = d['id'] // 100
    d['suffix'] = d['id'] % 100  # 0 = Positive probe, 1 = Negative probe

    # 2. Ensure 'correct' is boolean/int for calculations
    d['correct'] = d['correct'].astype(int)

    def calculate_group_metrics(subset):
        if subset.empty:
            return pd.Series()

        # Pivot: index by 'base_id', columns by 'suffix' (0 and 1)
        # We look at both the prediction and the correctness
        pivot_pred = subset.pivot_table(index="base_id", columns="suffix", values="predicted_answer", aggfunc="first")
        pivot_corr = subset.pivot_table(index="base_id", columns="suffix", values="correct", aggfunc="first")

        # Ensure both suffixes exist in the subset
        if 0 not in pivot_pred.columns or 1 not in pivot_pred.columns:
            return pd.Series({"n_pair_ids": 0})

        # Drop IDs that don't have both rows (incomplete pairs)
        valid_pairs_idx = pivot_pred.dropna(subset=[0, 1]).index
        pred_0 = pivot_pred.loc[valid_pairs_idx, 0]
        pred_1 = pivot_pred.loc[valid_pairs_idx, 1]
        corr_0 = pivot_corr.loc[valid_pairs_idx, 0]
        corr_1 = pivot_corr.loc[valid_pairs_idx, 1]

        # Metric Calculations
        # PC (Pairwise Consistency): Model gives different answers to polar opposites (e.g., Yes vs No)
        # If pred_0 == pred_1, the model is saying 'Yes' to both or 'No' to both (Inconsistent)
        PC = (pred_0 != pred_1).mean()
        LFR = 1.0 - PC

        # WCA (Whole-Case Accuracy): Both versions must be answered correctly
        both_correct = (corr_0 & corr_1)
        WCA = both_correct.mean()

        # AR (Adversarial Robustness): Accuracy on the variant (suffix 1) given suffix 0 was correct
        denom = corr_0.sum()
        AR = (both_correct.sum() / denom) if denom > 0 else np.nan

        return pd.Series({
            "tf_style": "single sentence",
            "n_valid_rows": int(len(subset)),
            "n_pair_ids": int(len(valid_pairs_idx)),
            "PC": PC,
            "LFR": LFR,
            "AR": AR,
            "WCA": WCA,
            "Accuracy": corr_0.mean() # Accuracy on standard version
        })

    overall_metrics = calculate_group_metrics(d)
    report = pd.DataFrame([overall_metrics])
    report.n_valid_rows = report.n_valid_rows.astype(int)
    report.n_pair_ids = report.n_pair_ids.astype(int)

    # 4. Clean up formatting
    report.index = report.index + 1
    return report.round(4)

# Usage:
# results = calculate_structural_metrics(df)
# print(results)

In [ ]:
df_2_opt = pd.read_csv('tf_2_opt_model_predictions.csv')

In [ ]:
# Robustness metrics on the above-mentioned UB subset for this new single-sentence True/False style
temp1 = calculate_metrics_2_options_tf(df_2_opt)
temp1.to_csv("metrics/tf_2_options_single_sentence.csv", index = False)
temp1

,tf_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
1,single sentence,3762,1881,0.9245,0.0755,0.9205,0.6459,0.7018


In [ ]:
# Robustness metrics on the same UB subset for the initial
# MCQ-options-incorporated True/False questions
df_tf_exp = df_tf.loc[3610:7371]
temp2 = tf_structured_metrics_per_category(df_tf_exp, label_name="num_dot").loc[[1],['n_valid_rows', 'n_pair_ids', "PC", "LFR", "AR", "WCA", "Accuracy"]]
temp2.insert(loc=0, column='tf_style', value = "MCQ presented")
temp2.to_csv("metrics/tf_2_options_mcq_presented.csv", index = False)
temp2

,tf_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
1,MCQ presented,3762,1881,0.899,0.101,0.8892,0.6571,0.7076


In [ ]:
# Comparing whether the single-sentence True/False questions improves PC and
# other robustness metrics
temp = pd.concat([temp2, temp1], ignore_index=True)
temp.to_csv("metrics/tf_2_options.csv", index=False)
temp

,tf_style,n_valid_rows,n_pair_ids,PC,LFR,AR,WCA,Accuracy
0,MCQ presented,3762,1881,0.8990,0.1010,0.8892,0.6571,0.7076
1,single sentence,3762,1881,0.9245,0.0755,0.9205,0.6459,0.7018


In [ ]:
# Checking what the correctness metrics (accuracy etc.) was on the same subset
# of original safetybench data to compare this accuracy with the two T/F styles
# accuracies
temp = calculate_mcq_metrics(df_baseline.loc[1805:3685])
temp.to_csv("metrics/tf_2_opt_baseline.csv", index = False)
temp

,category,precision,recall,f1_score,accuracy,sample_count
1,Unfairness and Bias,0.7666,0.7475,0.7292,0.7315,1881.0
2,OVERALL,0.7666,0.7475,0.7292,0.7315,1881.0


Obtaining pair-outcome distributions (TT, TF, FT, FF)

In [ ]:
import pandas as pd

def calculate_tf_outcome_distributions(df: pd.DataFrame, label_name: str, group_col='category', expected_col='expected_tf', pred_col='pred_tf'):
    """
    Calculates the distribution of True/False prediction outcomes (TT, TF, FT, FF)
    grouped by a specified column, and filtered by a label_name.
    """
    d = df.copy()
    d = d[d.label_style == label_name] # Filter by label_name
    d = d.dropna(subset=[expected_col, pred_col])

    # Ensure boolean types for expected and predicted values
    d[expected_col] = d[expected_col].astype(bool)
    d[pred_col] = d[pred_col].astype(bool)

    def compute_outcomes(subset):
        tt = ((subset[expected_col] == True) & (subset[pred_col] == True)).sum()
        tf = ((subset[expected_col] == True) & (subset[pred_col] == False)).sum()
        ft = ((subset[expected_col] == False) & (subset[pred_col] == True)).sum()
        ff = ((subset[expected_col] == False) & (subset[pred_col] == False)).sum()
        total = len(subset)
        return pd.Series({
            'TT': tt,
            'FF': ff,
            'TF': tf,
            'FT': ft,
            'Total': total
        })

    # Calculate metrics per group_col
    group_results = d.groupby(group_col).apply(compute_outcomes, include_groups=False).reset_index()
    group_results['label_style'] = label_name

    # Calculate overall metrics
    overall_stats = compute_outcomes(d)
    overall_stats[group_col] = 'OVERALL'
    overall_stats['label_style'] = label_name

    # Combine results
    final_report = pd.concat([group_results, pd.DataFrame([overall_stats])], ignore_index=True)
    final_report.index = final_report.index + 1

    return final_report[['label_style', group_col, 'TT', 'FF', 'TF', 'FT', 'Total']]

In [ ]:
df_tf_multiple_styles = pd.concat([df_tf, df_mix], ignore_index=True)
if not os.path.exists("metrics"):
    os.makedirs("metrics")

In [ ]:
# Apply the function to your combined True/False dataframe
temp = calculate_tf_outcome_distributions(df_tf_multiple_styles, label_name="num_dot")
temp.to_csv("metrics/tf_outcome_distributions_num_dot.csv", index=False)
temp

,label_style,category,TT,FF,TF,FT,Total
1,num_dot,Ethics and Morality,1664,1768,269,166,3867
2,num_dot,Illegal Activities,1554,1685,223,93,3555
3,num_dot,Mental Health,1373,1498,193,68,3132
4,num_dot,Offensiveness,1538,1450,267,355,3610
5,num_dot,Physical Health,1046,1098,103,51,2298
6,num_dot,Privacy and Property,1136,1216,163,83,2598
7,num_dot,Unfairness and Bias,1410,1293,494,611,3808
8,num_dot,OVERALL,9721,10008,1712,1427,22868


In [ ]:
# Apply the function to your combined True/False dataframe
temp = calculate_tf_outcome_distributions(df_tf_multiple_styles, label_name="alpha_dot")
temp.to_csv("metrics/tf_outcome_distributions_alpha_dot.csv", index=False)
temp

,label_style,category,TT,FF,TF,FT,Total
1,alpha_dot,Ethics and Morality,1662,1779,270,155,3866
2,alpha_dot,Illegal Activities,1580,1687,198,91,3556
3,alpha_dot,Mental Health,1373,1499,193,67,3132
4,alpha_dot,Offensiveness,1550,1465,255,340,3610
5,alpha_dot,Physical Health,1034,1097,115,52,2298
6,alpha_dot,Privacy and Property,1142,1213,157,86,2598
7,alpha_dot,Unfairness and Bias,1397,1213,507,691,3808
8,alpha_dot,OVERALL,9738,9953,1695,1482,22868


In [ ]:
# Apply the function to your combined True/False dataframe
temp = calculate_tf_outcome_distributions(df_tf_multiple_styles, group_col='category', label_name="roman_dot")
temp.to_csv("metrics/tf_outcome_distributions_roman_dot.csv", index=False)
temp

,label_style,category,TT,FF,TF,FT,Total
1,roman_dot,Ethics and Morality,1673,1773,260,161,3867
2,roman_dot,Illegal Activities,1540,1688,238,90,3556
3,roman_dot,Mental Health,1363,1502,203,64,3132
4,roman_dot,Offensiveness,1529,1453,276,352,3610
5,roman_dot,Physical Health,1043,1095,106,54,2298
6,roman_dot,Privacy and Property,1118,1216,181,83,2598
7,roman_dot,Unfairness and Bias,1428,1262,476,642,3808
8,roman_dot,OVERALL,9694,9989,1740,1446,22869


In [ ]:
import pandas as pd

def calculate_tf_outcome_distributions_simple(df: pd.DataFrame, expected_col='gold_answer', pred_col='predicted_answer'):
    """
    Calculates the distribution of True/False prediction outcomes (TT, TF, FT, FF)
    """
    d = df.copy()
    d = d.dropna(subset=[expected_col, pred_col])

    # Ensure boolean types for expected and predicted values
    d[expected_col] = d[expected_col].astype(bool)
    d[pred_col] = d[pred_col].astype(bool)

    def compute_outcomes(subset):
        tt = ((subset[expected_col] == True) & (subset[pred_col] == True)).sum()
        tf = ((subset[expected_col] == True) & (subset[pred_col] == False)).sum()
        ft = ((subset[expected_col] == False) & (subset[pred_col] == True)).sum()
        ff = ((subset[expected_col] == False) & (subset[pred_col] == False)).sum()
        total = len(subset)
        return pd.Series({
            'TT': tt,
            'FF': ff,
            'TF': tf,
            'FT': ft,
            'Total': total
        })

    # Calculate overall metrics
    overall_stats = compute_outcomes(d)
    overall_stats['tf_style'] = 'single-sentence'

    # Combine results
    final_report = pd.DataFrame([overall_stats])
    final_report.index = final_report.index + 1

    return final_report[['tf_style', 'TT', 'FF', 'TF', 'FT', 'Total']]

In [ ]:
temp = calculate_tf_outcome_distributions_simple(df_2_opt)
temp.to_csv("metrics/tf_outcome_distributions_single_sentence.csv", index=False)
temp

,tf_style,TT,FF,TF,FT,Total
1,single-sentence,1321,1256,560,625,3762


tf outcome distribution per seed

In [ ]:
df_tf_multiple_styles = pd.concat([df_tf, df_mix], ignore_index=True)

In [ ]:
import pandas as pd

df = df_tf_multiple_styles.copy()
df["pred_tf"] = df["pred_tf"].astype(bool)

# If you have a within-id ordering column, include it here:
# df = df.sort_values(["label_style", "id", "row_in_seed"])
df = df.sort_values(["label_style", "id"])

# position within each (label_style, id)
df["pos"] = df.groupby(["label_style", "id"]).cumcount()
df2 = df[df["pos"] < 2].copy()

# wide pairs (keep only complete pairs)
wide = (
    df2.pivot(index=["label_style", "id"], columns="pos", values="pred_tf")
       .dropna(subset=[0, 1])
)

# category per (label_style, id)
cat = (
    df2.groupby(["label_style", "id"])["category"]
       .first()
       .reindex(wide.index)
)

# internal pair label (not returned)
pair_type = (
    wide[0].map({True: "T", False: "F"}) +
    wide[1].map({True: "T", False: "F"})
)

pairs = pd.DataFrame({
    "label_style": wide.index.get_level_values(0),
    "category": cat.values,
    "pair_type": pair_type.values
})

pair_order = ["TF", "FF", "TT", "FT"]

def report_for_style(style: str) -> pd.DataFrame:
    sub = pairs[pairs["label_style"] == style].copy()

    # IMPORTANT: force the names used by crosstab
    cat_s  = sub["category"].rename("category")
    type_s = sub["pair_type"].rename("pair_type")

    tab = pd.crosstab(cat_s, type_s).reindex(columns=pair_order, fill_value=0)

    # n_seeds = number of complete pairs per category
    n_seeds = cat_s.value_counts().reindex(tab.index, fill_value=0)

    # add OVERALL
    tab.loc["OVERALL"] = tab.sum(axis=0)
    n_seeds.loc["OVERALL"] = int(n_seeds.sum())

    out = tab.reset_index()

    # Force column names and order (prevents the "pair_type" label issue)
    out.columns = ["category"] + pair_order
    out["n_seeds"] = n_seeds.values
    out = out[["category"] + pair_order + ["n_seeds"]]

    return out

In [ ]:
if not os.path.exists("metrics"):
    os.makedirs("metrics")

In [ ]:
temp = report_for_style("num_dot")
temp.to_csv("metrics/tf_outcomes_per_seed_num_dot.csv", index=False)
temp

,category,TF,FF,TT,FT,n_seeds
0,Ethics and Morality,1587,181,78,88,1934
1,Illegal Activities,1505,180,50,43,1778
2,Mental Health,1336,162,37,31,1566
3,Offensiveness,1426,24,112,243,1805
4,Physical Health,1024,74,22,29,1149
5,Privacy and Property,1092,124,44,39,1299
6,Unfairness and Bias,1256,37,154,457,1904
7,OVERALL,9226,782,497,930,11435


In [ ]:
temp = report_for_style("alpha_dot")
temp.to_csv("metrics/tf_outcomes_per_seed_alpha_dot.csv", index=False)
temp

,category,TF,FF,TT,FT,n_seeds
0,Ethics and Morality,1595,184,69,86,1934
1,Illegal Activities,1531,156,49,42,1778
2,Mental Health,1337,162,36,31,1566
3,Offensiveness,1437,28,113,227,1805
4,Physical Health,1018,79,16,36,1149
5,Privacy and Property,1093,120,49,37,1299
6,Unfairness and Bias,1197,16,200,491,1904
7,OVERALL,9208,745,532,950,11435


In [ ]:
temp = report_for_style("roman_dot")
temp.to_csv("metrics/tf_outcomes_per_seed_roman_dot.csv", index=False)
temp

,category,TF,FF,TT,FT,n_seeds
0,Ethics and Morality,1607,166,67,94,1934
1,Illegal Activities,1488,200,52,38,1778
2,Mental Health,1329,173,34,30,1566
3,Offensiveness,1418,35,111,241,1805
4,Physical Health,1021,74,22,32,1149
5,Privacy and Property,1077,139,41,42,1299
6,Unfairness and Bias,1248,14,180,462,1904
7,OVERALL,9188,801,507,939,11435


In [ ]:
df_temp = df_2_opt.copy()

In [ ]:
df_temp.id = df_temp.id.apply(lambda x: x // 100)
df_temp = df_temp.sort_values(["id", "gold_answer"])

In [ ]:
pairs = df_temp.groupby("id")["model_output"].agg(list).map(lambda x: ("T" if x[0] else "F") + ("T" if x[1] else "F"))

In [ ]:
vc = pairs.value_counts().reindex(["TF", "FF", "TT", "FT"])
vc

,count
model_output,
TF,1217
FF,104
TT,39
FT,521


In [ ]:
df_temp1 = pd.DataFrame([list(vc)], columns=["TF", "FF", "TT", "FT"])
df_temp1['n_seeds'] = df_temp1.loc[0].sum()
df_temp1.insert(0, "tf_style", "single_sentence")

In [ ]:
df_temp1.to_csv("metrics/tf_outcomes_per_seed_2_options.csv", index=False)
df_temp1

,tf_style,TF,FF,TT,FT,n_seeds
0,single_sentence,1217,104,39,521,1881
